# 04. Итоговая оценка

## Итоговая оценка: качество, недискриминационность, скорость

In [ ]:

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

from src.utils import (
    evaluate_at_threshold,
    pick_threshold_for_target_recall,
    group_rates_at_threshold,
    disparity_summary,
    measure_inference_latency_ms,
)

DATA_PATH = Path("../data/GiveMeSomeCredit-training.csv")  # поменяй под себя
df = pd.read_csv(DATA_PATH)
for c in df.columns:
    if c.lower().startswith("unnamed"):
        df = df.drop(columns=[c])

TARGET = "SeriousDlqin2yrs"
df = df[(df["age"] >= 18) & (df["age"] <= 120)].copy()
df["age_group"] = pd.cut(df["age"], bins=[17,24,34,44,54,64,200],
                         labels=["18-24","25-34","35-44","45-54","55-64","65+"], right=True)

train_df, temp_df = train_test_split(df, test_size=0.30, stratify=df[TARGET], random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.50, stratify=temp_df[TARGET], random_state=42)

X_train = train_df.drop(columns=[TARGET, "age_group"], errors="ignore")
y_train = train_df[TARGET].astype(int)
X_valid = valid_df.drop(columns=[TARGET, "age_group"], errors="ignore")
y_valid = valid_df[TARGET].astype(int)
X_test  = test_df.drop(columns=[TARGET, "age_group"], errors="ignore")
y_test  = test_df[TARGET].astype(int)

model = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42))
])

model.fit(X_train, y_train)

proba_valid = model.predict_proba(X_valid)[:, 1]
proba_test  = model.predict_proba(X_test)[:, 1]

# Порог по полноте
THRESHOLD_FINAL = pick_threshold_for_target_recall(y_valid, proba_valid, target_recall=0.80)

# Качество
print("=== КАЧЕСТВО (TEST) ===")
print("ROC-AUC:", roc_auc_score(y_test, proba_test))
print("PR-AUC :", average_precision_score(y_test, proba_test))
print("Brier :", brier_score_loss(y_test, proba_test))
print("Порог :", THRESHOLD_FINAL)

ev = evaluate_at_threshold(y_test, proba_test, threshold=THRESHOLD_FINAL)
print("\n=== МЕТРИКИ ПРИ ПОРОГЕ ===")
print(ev)

# Недискриминационность по возрастным группам
gt = group_rates_at_threshold(y_test, proba_test, group=test_df["age_group"], threshold=THRESHOLD_FINAL)
print("\n=== ТАБЛИЦА ПО ВОЗРАСТНЫМ ГРУППАМ ===")
print(gt.to_string(index=False))

disp = disparity_summary(gt)
print("\n=== СВОДКА РАЗЛИЧИЙ МЕЖДУ ГРУППАМИ ===")
for k, v in disp.items():
    print(k, "=", v)

# Скорость
lat = measure_inference_latency_ms(model, X_test, n_runs=30, batch_size=1000)
print("\n=== СКОРОСТЬ ===")
print(lat)
